In [ ]:
!pip install langchain-community         

In [ ]:
!pip install faiss-cpu 

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.document_loaders.csv_loader import CSVLoader     
from langchain_community.embeddings import HuggingFaceEmbeddings 
from langchain.vectorstores import FAISS      
from charset_normalizer import from_path
import pandas as pd    
from langchain_community.embeddings import HuggingFaceEmbeddings  
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA 
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline  
from langchain_community.llms import HuggingFacePipeline  

In [6]:
csv_path = "/kaggle/input/qa-dataset/codebasics_faqs.csv" 
  
# 1) Detect encoding
enc = (from_path(csv_path).best() or from_path(csv_path).first()).encoding or "latin1" 

# 2) Read CSV (handle both new/old pandas) 
read_kwargs = {"encoding": enc} 
try:
    df = pd.read_csv(csv_path, on_bad_lines="skip", **read_kwargs)  
except TypeError:  
    # for older pandas 
    df = pd.read_csv(csv_path, error_bad_lines=False, **read_kwargs) 
   
# 3) Turn into Documents
try:  
    from langchain_community.document_loaders import DataFrameLoader 
except ImportError:  
    # Fallback if langchain_community isn't available
    from langchain.document_loaders import DataFrameLoader

loader = DataFrameLoader(df, page_content_column="prompt")  # <- use your text column 
docs = loader.load() 
 
docs[:2]  # quick peek  

[Document(metadata={'response': 'Yes, this is the perfect bootcamp for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in your current job or business using data.'}, page_content='I have never done programming in my life. Can I take this bootcamp?'),
 Document(metadata={'response': 'Till now 9000 + learners have benefitted from the quality of our courses. You can check the review section and also we have attached their LinkedIn profiles so that you can connect with them and ask directly.'}, page_content='Why should I trust Codebasics?')]

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={"device": "cpu"})
vectordb = FAISS.from_documents(documents=docs, embedding=embeddings)        